## Spark + Yelp Exercise, pt. 2
Spark exercise leveraging the Yelp Academic Dataset to improve familiarity with PySpark and work on basics of Spark SQL. This exercise will leverage `spark.sql()` calls to query datasets and use fewest number of steps.

`%%time` functions included, but results may vary by operating system configuration.
_________________________

First, spin up a Spark session.

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName('My First Spark application') \
    .getOrCreate() 

sc = spark.sparkContext

Load datasets.

In [2]:
# business = spark.read.json('../assets/yelp_academic/yelp_academic_dataset_business.json.gz')
# checkin = spark.read.json('../assets/yelp_academic/yelp_academic_dataset_checkin.json.gz')
# review = spark.read.json('../assets/yelp_academic/yelp_academic_dataset_review.json.gz')
# tip = spark.read.json('../assets/yelp_academic/yelp_academic_dataset_tip.json.gz')
# user = spark.read.json('../assets/yelp_academic/yelp_academic_dataset_user.json.gz')

business = spark.read.json('data/yelp_academic/yelp_academic_dataset_business.json.gz')
checkin = spark.read.json('data/yelp_academic/yelp_academic_dataset_checkin.json.gz')
review = spark.read.json('data/yelp_academic/yelp_academic_dataset_review.json.gz')
tip = spark.read.json('data/yelp_academic/yelp_academic_dataset_tip.json.gz')
user = spark.read.json('data/yelp_academic/yelp_academic_dataset_user.json.gz')

Create temp views for DataFrames (act like hive table). Data will only be cached after `.count` call.

In [3]:
business.createOrReplaceTempView("business")
checkin.createOrReplaceTempView("checkin")
tip.createOrReplaceTempView("tip")
review.createOrReplaceTempView("review")
user.createOrReplaceTempView("user")

### Exercise Questions:

#### How many users have more than 500 fans?

In [4]:
%%time

query = """SELECT * FROM user WHERE fans > 500"""

spark.sql(query).count()

CPU times: user 1.27 ms, sys: 1.94 ms, total: 3.21 ms
Wall time: 17.3 s


185

#### How many businesses from Madison, Wisconsin are represented in the dataset?

In [5]:
%%time

query = """
SELECT * 
FROM business 
WHERE city ='Madison' AND state='WI'
"""

spark.sql(query).count()

CPU times: user 1.4 ms, sys: 0 ns, total: 1.4 ms
Wall time: 1.4 s


3493

#### How many users have contributed only one review?

In [6]:
%%time

query = """
SELECT COUNT(DISTINCT(user_id)) as users
FROM user
WHERE review_count = 1
"""

query = spark.sql(query).show()

+------+
| users|
+------+
|298779|
+------+

CPU times: user 2.97 ms, sys: 414 µs, total: 3.38 ms
Wall time: 19.6 s


#### How many businesses are north of the Mason-Dixon Line?

https://en.wikipedia.org/wiki/Mason%E2%80%93Dixon_line

In [7]:
%%time

# Assuming Mason-Dixon Latitude of 39º43'19.92216" N (39.7222006) 
# and boundary end at Longitude 075º47'19.93851' W (075.78859403055556)
# (converted to decimals using https://www.directionsmag.com/site/latlong-converter/)

query = """
SELECT COUNT(DISTINCT(business_id)) as businesses
FROM business
WHERE latitude > 39.7222006 and longitude > -75.78859403055556
"""

query = spark.sql(query).show()

+----------+
|businesses|
+----------+
|      9244|
+----------+

CPU times: user 1.76 ms, sys: 0 ns, total: 1.76 ms
Wall time: 1.79 s


#### Which non-US states are represented in the dataset?

- List the distinct state codes that are not in a list of US codes.
- It is assumed that the state codes are only the traditional US states, and not any commonwealth or territories.
- Helpful list of state abbreviations for easy c/p: https://gist.github.com/JeffPaine/3083347

In [8]:
%%time

query = """
SELECT DISTINCT(state) as non_us
FROM business
WHERE state NOT IN ("AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY")
"""

query = spark.sql(query).show()

+------+
|non_us|
+------+
|   BAS|
|    QC|
|    BC|
|   XGL|
|   XGM|
|   DUR|
|   DOW|
|    ON|
|    AB|
|   CON|
|   XWY|
+------+

CPU times: user 976 µs, sys: 767 µs, total: 1.74 ms
Wall time: 1.39 s


#### Which US states are represented in the dataset? Use full names of states.

In [9]:
%%time
import pandas as pd

# Pull query
states = spark.sql("SELECT DISTINCT state FROM business").toPandas()

# Generate list of name-to-code conversions
state_list = {"AL":"Alabama","AK":"Alaska","AZ":"Arizona","AR":"Arkansas","CA":"California","CO":"Colorado","CT":"Connecticut","DE":"Delaware","FL":"Florida","GA":"Georgia","HI":"Hawaii","ID":"Idaho","IL":"Illinois","IN":"Indiana","IA":"Iowa","KS":"Kansas","KY":"Kentucky","LA":"Louisiana","ME":"Maine","MD":"Maryland","MA":"Massachusetts","MI":"Michigan","MN":"Minnesota","MS":"Mississippi","MO":"Missouri","MT":"Montana","NE":"Nebraska","NV":"Nevada","NH":"New Hampshire","NJ":"New Jersey","NM":"New Mexico","NY":"New York","NC":"North Carolina","ND":"North Dakota","OH":"Ohio","OK":"Oklahoma","OR":"Oregon","PA":"Pennsylvania","RI":"Rhode Island","SC":"South Carolina","SD":"South Dakota","TN":"Tennessee","TX":"Texas","UT":"Utah","VT":"Vermont","VA":"Virginia","WA":"Washington","WV":"West Virginia","WI":"Wisconsin","WY":"Wyoming"}

# Replace state code list with state names
states.replace(state_list, inplace=True)

# Reduce to only US states, due to provinces listed in previous exercise not being covered
states = states[states['state'].isin(list(state_list.values()))]
print("There are " + str(len(states)) + " states represented in the dataset.")
states

There are 25 states represented in the dataset.
CPU times: user 250 ms, sys: 24.8 ms, total: 275 ms
Wall time: 1.62 s


,state
0,Arizona
1,South Carolina
3,New Jersey
4,Virginia
7,Nevada
8,Wisconsin
9,California
10,Nebraska
11,Connecticut
12,North Carolina


#### What is the text of the funniest review?

In [10]:
%%time

query = """
SELECT text,funny 
FROM review 
ORDER BY funny DESC
LIMIT 1
"""

query = spark.sql(query).toPandas().text[0]
print(query, "\n")

Flew to Arizona a few months ago to try this. Disappointed that this place closed after Gordon Ramsay's return. Their food was one of the best food I had in my whole entire life!! Their caesar salad was great, sauce is very good. Their pizza was THE BEST I HAD IN MY WHOLE ENTIRE LIFE. From the garlic crust to the extremely cheesy topping, the pizza was absolutely outstanding. Wish I could come back but its now closed. 

CPU times: user 3.05 ms, sys: 7.61 ms, total: 10.7 ms
Wall time: 58.3 s


#### Which company was the funniest review from?

In [11]:
%%time

query = """
SELECT business.name
FROM review

LEFT JOIN business
ON review.business_id = business.business_id

WHERE review.funny IN (SELECT MAX(review.funny)
                       FROM review)
"""

query = spark.sql(query).show()

+--------------------+
|                name|
+--------------------+
|Amy's Baking Company|
+--------------------+

CPU times: user 7.13 ms, sys: 4.47 ms, total: 11.6 ms
Wall time: 1min 36s


#### Which review(s) has/have the most words?

In [12]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf

# assign a UDF to count character length
char_len = udf(lambda x: len(x), IntegerType())
spark.udf.register('char_len', char_len)

<function __main__.<lambda>(x)>

In [13]:
%%time

# In the process of trying to solve this, I struggled with leveraging
# `WHERE char_len(text) = (SELECT MAX(char_len(text)) FROM review)`
# as one would in standard SQL. After multiple attempts, I landed on
# this f-string workaround - though not my favorite solution for 
# efficiency as it iterates through a large dataset twice.

query1 = "(SELECT MAX(char_len(text)) AS max_len FROM review)"
query1 = spark.sql(query1).toPandas()
max_len = str(query1['max_len'].iloc[0])
max_len

# run query leveraging UDF
query = f"""
SELECT text
       ,cool
       ,funny
       ,stars
       ,useful
FROM review
WHERE char_len(text) = {max_len}
"""

query = spark.sql(query).limit(5).toPandas()
query

CPU times: user 13.3 ms, sys: 2.84 ms, total: 16.1 ms
Wall time: 1min 17s


,text,cool,funny,stars,useful
0,This place is awful. If quick and dirty brunch...,2,8,1.0,13
1,My husband and I were so overly disappointed i...,0,1,1.0,1
2,My review will focus on the Arizona State Fair...,2,2,4.0,1
3,"Brides, stop your wedding venue searching and ...",0,0,5.0,2
4,"Paul's Yelp Elite Rating: 21 Stars, er, 10 Sta...",7,3,5.0,9


#### What are the names of the top 10 users who provided the most tips?

In [14]:
%%time

query = """
SELECT a.user_id , b.name, a.tip_cnt
FROM (SELECT user_id, count(user_id) as tip_cnt 
      FROM tip 
      GROUP BY user_id
      ORDER BY tip_cnt DESC) a
LEFT JOIN  user b
ON a.user_id = b.user_id  
ORDER BY a.tip_cnt DESC
LIMIT 10
"""

query = spark.sql(query).toPandas()
query

CPU times: user 5.56 ms, sys: 895 µs, total: 6.45 ms
Wall time: 29.8 s


,user_id,name,tip_cnt
0,mkbx55W8B8aPLgDqex7qgg,Momo,2439
1,CxDOIDnH8gp9KXzpBHJYXw,Jennifer,1598
2,6ZC-0LfOAGwaFc5XPke74w,Samantha,1509
3,0tvCcnfJnSs55iB6mqPk3w,Daniel,1376
4,eZfHm0qI8A_HfvXScwIYsg,Christie,1352
5,O8eDScRAg6ae0l9Bc24uMA,May,1255
6,8DGFWco9VeBAxjqsuh1aSw,Kurdy,1178
7,WJKocp9RE0KatUwh3_DkGg,Anthony,1161
8,2EuPAGalYnP7eSxPgFCNDg,Shirley,1154
9,QPJJohtGqkMkaN0Gt3TRIg,Cherrie,1017


#### List the names, number of reviews of businesses in Arizona and total number of reviews of the top 5 users (as determined by who has created the most number of reviews of businesses in Arizona). Include a column that shows the percentage of reviews that are of businesses from Arizona.

In [15]:
from pyspark.sql.types import IntegerType, FloatType

In [16]:
%%time

query = """
SELECT user.name AS name
       ,az_bus.o_az_count AS az_cnt
       ,user.review_count AS tot_cnt
       ,(az_bus.o_az_count/user.review_count)*100 as pct
FROM (SELECT r.user_id
             ,count(r.user_id) o_az_count 
      FROM business AS bus 
      RIGHT JOIN review  AS r
      ON bus.business_id = r.business_id
      WHERE bus.state = 'AZ'
      GROUP BY r.user_id
      ORDER BY o_az_count DESC
      ) az_bus
RIGHT JOIN user
ON az_bus.user_id = user.user_id
ORDER BY az_bus.o_az_count DESC
LIMIT 5
"""

query = spark.sql(query).toPandas()
query

,name,az_cnt,tot_cnt,pct
0,Brad,1637,1642,99.695493
1,Karen,1559,2340,66.623932
2,Jennifer,1250,1929,64.800415
3,Gabi,1151,1932,59.575569
4,Judy,1059,1193,88.767812


______________________
<div style="text-align: right"><sub>Exercise adapted and modified from UMSI homework assignment for SIADS 516.</sub></div>